In [4]:
import numpy as np
import cv2
import os
import torch
from tqdm import tqdm
import config
from model import model

In [5]:
# set the computation device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# load the model and the trained weights
model = model().to(device)
model.load_state_dict(torch.load('Data/checkpoints/fasterrcnn_resnet50_fpn.pth'))

DIR_TEST = 'Test'
test_images = os.listdir(DIR_TEST)
print(f"Validation instances: {len(test_images)}")

detection_threshold = config.PREDICTION_THRES
model.eval()
with torch.no_grad():
    for i, image in tqdm(enumerate(test_images), total=len(test_images)):
        orig_image = cv2.imread(f"{DIR_TEST}/{test_images[i]}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
        # make the pixel range between 0 and 1
        image /= 255.0
        image = np.transpose(image, (2, 0, 1)).astype(np.float)
        image = torch.tensor(image, dtype=torch.float).cuda()
        image = torch.unsqueeze(image, 0)
        cpu_device = torch.device("cpu")
        outputs = model(image)
        
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        if len(outputs[0]['boxes']) != 0:
            for counter in range(len(outputs[0]['boxes'])):
                boxes = outputs[0]['boxes'].data.numpy()
                scores = outputs[0]['scores'].data.numpy()
                boxes = boxes[scores >= detection_threshold].astype(np.int32)
                draw_boxes = boxes.copy()
                
            for box in draw_boxes:
                cv2.rectangle(orig_image,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            (0, 0, 255), 3)
                cv2.putText(orig_image, 'PotHole', 
                            (int(box[0]), int(box[1]-5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 
                            2, lineType=cv2.LINE_AA)
            cv2.imwrite(f"test_predictions/{test_images[i]}", orig_image,)
print('TEST PREDICTIONS COMPLETE')


Validation instances: 4336


  0%|          | 0/4336 [00:00<?, ?it/s]C:\Users\Darke\AppData\Local\Temp/ipykernel_20112/2044359032.py:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  image = np.transpose(image, (2, 0, 1)).astype(np.float)
D:\Python\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 4336/4336 [54:07<00:00,  1.34it/s]  

TEST PREDICTIONS COMPLETE


In [6]:
from os.path import isfile, join
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    # print(files)
    #for sorting the file names properly
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [3]:
pathIn= 'test_predictions/'
pathOut = 'video.avi'
fps = 25.0
convert_frames_to_video(pathIn,pathOut,fps)

test_predictions/G0011476.JPG
test_predictions/G0011523.JPG
test_predictions/G0011524.JPG
test_predictions/G0011562.JPG
test_predictions/G0011585.JPG
test_predictions/G0011587.JPG
test_predictions/G0011601.JPG
test_predictions/G0011602.JPG
test_predictions/G0011603.JPG
test_predictions/G0011614.JPG
test_predictions/G0011616.JPG
test_predictions/G0011661.JPG
test_predictions/G0011662.JPG
test_predictions/G0011664.JPG
test_predictions/G0011677.JPG
test_predictions/G0011678.JPG
test_predictions/G0011705.JPG
test_predictions/G0011769.JPG
test_predictions/G0011772.JPG
test_predictions/G0011773.JPG
test_predictions/G0011876.JPG
test_predictions/G0011878.JPG
test_predictions/G0011881.JPG
test_predictions/G0011966.JPG
test_predictions/G0011969.JPG
test_predictions/G0011970.JPG
test_predictions/G0011978.JPG
test_predictions/G0011981.JPG
test_predictions/G0011982.JPG
test_predictions/G0011986.JPG
test_predictions/G0011988.JPG
test_predictions/G0011989.JPG
test_predictions/G0012011.JPG
test_predi